# Let's generate some text

No training here, only fun stuff.

In [1]:
from fastai_old.text import *
import html
import spacy 

#spacy.load('en')

Note that the itos/stoi have to match the model you're loading! Obvious but, you know. Remember.

When we created the language model, we give it a path (in this case `data_inf/custom_lm/`). Then when we save the language model, it saves the data in `path/models/`. So we're pulling the models from `data_inf/custom_lm/models/`.

If we want to load just the pre-trained data model in the same way, we need to load it up in the generating file and save it before it gets trained. These should be located in `data_inf/pretrained/`.

Also I have set the batch size (`bs`) to 1 to avoid errors of processing the data. Since we're *not doing any training* this doesn't matter.

In [4]:
LM_PATH = Path('data_inf/custom_lm/')
# LM_PATH = Path('data_inf/custom_lm/test_vocab/')
# LM_PATH = Path('data_inf/pretrained/')
LM_PATH.mkdir(exist_ok=True)

em_sz,nh,nl = 400,1150,3

wd=1e-7
bptt=70
bs=1
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

# itos_name = 'itos_wt103.pkl'
# model_name = 'pretrained'

# itos_name = 'itos_imdb.pkl'
# model_name = 'lm_last_ft'

# itos_name = 'itos_alice_nop.pkl'
# model_name = 'alice_nop_lm_30epochs'

# itos_name = 'itos_alice_processed.pkl'
# model_name = 'alice_processed_lm_30epochs'

#itos_name = 'itos_pop_quotes.pkl'
#model_name = 'pop_quotes_lm_30epochs'

itos_name = 'itos_pop_quotesnotebook.pkl'
model_name = 'pop_quotesnotebook_lm_30epochs'

itos2 = pickle.load((LM_PATH/itos_name).open('rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

trn_lm = np.array([list([0,0])])
val_lm = np.array([list([0,0])])

vs=len(itos2)

trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(LM_PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.load(model_name)
m = learner.model
vs

2121

In [5]:
def generate_text(m, s, l=20):
    m[0].bs=1  # Set batch size to 1
    m.eval()  # Turn off dropout
    m.reset()  # Reset hidden state
    m[0].bs=bs  # Put the batch size back to what it was

    ss = s.lower().split()
    si = [stoi2[w] for w in ss]
    t = torch.autograd.Variable(torch.cuda.LongTensor(np.array([si])))
    
    res,*_ = m(t)

    print(s, end=' ')
    for i in range(l):
#         n = res[-1].topk(5)[1]  # top word
        n = torch.multinomial(res[-1].exp(), 2)  # drawing from probability distribution
        n = n[1] if n.data[0]==0 else n[0]
        print(itos2[int(n)], end=' ')
        res,*_ = m(n.unsqueeze(0).unsqueeze(0))  # sometimes need an extra .unsqueeze(0)
    print('...')

In [6]:
generate_text(m, "<bos>", l=60)

<bos> and better than anyone else remember them was a child of god and of great power and that great passion was 

 

 <bos> war is always not merely the name men gave to their mistakes but the passion and the passion which they brought to the idea <eos> 

 <bos> i can not educate myself when i m alone ...


For some reason drawing from the distribution (using multinomial) produces garbage for the pre-trained model. It's fine for the fine-tuned model. Generally drawing from the distribution is better and more interesting; using just the top word is repetitive and boring.

In [7]:
for i in range(5):
    print(i)
    generate_text(m, "<bos>", l=50)

0
<bos> if you do nt have time to read you do nt have the time or the tools to write simple as that nt are somewhere in between <eos> 
 <bos> nod to the nice girl the night before you start running to new machines and look at the people i ...
1
<bos> image is a human being with a black spot some shows the power of a black eye whatever is right whatever we actually are an idea and perhaps the victim is forgot how we do it <eos> 
 <bos> is it possible that the existence of space is entirely different ...
2
<bos> first a simple adventure in the abyss a few minutes before we begin smiling we see the old man at the sun then we throw ourselves into the darkness and he can see the place of happy heart <eos> 
 <bos> unless you see or you know you re a ...
3
<bos> progress is an irresistible desire to become a dreamer <eos> 

 <bos> we must learn to live from the moment it takes it out to be brave and vain and dare to forgive others for being wrong <eos> 
 <bos> to at the time the book expresse

In [8]:
for i in range(30):
    print(i)
    generate_text(m, "<bos>", l=50)

0
<bos> beginning its life as an ordinary person just as well <eos> 

 <bos> wanting to change is the way people feel the worst thing a person can be is to yourself <eos> 
 <bos> this is where it all begins everything starts here today <eos> 

 <bos> this was your ...
1
<bos> curl his hand but move forward him i suppose that s the thing about him so that he can spend his whole life <eos> 
 <bos> sell your soul and buy soul truer soul if you re interested <eos> 

 <bos> obtain the feeling of being unable to love anybody ...
2
<bos> are those who have never done anything right <eos> 

 <bos> saddest persons are more likely to be loved than to live <eos> 

 <bos> none of us really wants to bothered about anything wrong or not <eos> 
 <bos> forget your half blood half his sorrows curl your head ...
3
<bos> did you really like your work according to the fashion rather follow your most bravery <eos> 

 <bos> what is that is true what you call it the simple things <eos> 

 <bos> what is the p